In [ ]:
print('Hy')

import pandas as pd
import numpy as np
import matplotlib.pylab as plt

df = pd.read_csv("C:/Users/TRYMORE/Desktop/DataSets/creditdata.csv")
print('Columns:',df.columns)

#Separate continuos columns
num_cols = df.select_dtypes(include = ['number']).columns
print('Numerical Columns:', num_cols)

#Select categorical colums
cat_cols = df.select_dtypes(include = ['object', 'category']).columns
print('Categorical Columns:', cat_cols)

#Fill missing values for numerical variables
df[num_cols] = df[num_cols].fillna(df[num_cols].mean())

#Fill missing values for categorical variables
df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode())

#Fix data types once 
df = df.convert_dtypes()

#Drop column Customer ID since its not a variable
df.drop(columns = ['Customer_ID'],inplace = True)

#Label encording ordinal variables

from sklearn.preprocessing import LabelEncoder as ln
df['Level_of_Education'] = ln().fit_transform(df['Level_of_Education'])
df['Residence_Area'] = ln().fit_transform(df['Residence_Area'])
df['Tenure'] = ln().fit_transform(df['Tenure'])

#Norminal  variables Target encoding
#Install it with pip install category_encoders
import category_encoders as ce
#List the norminal variables
nominal_vars = ['Gender', 'Marital_Status', 'Employment_Status','Home_Ownership','Other_Solar_Product_purchase_on_installments']
#Initialize the encoder
target_encoder = ce.TargetEncoder(cols=nominal_vars)
#Fit and transform the data
df[nominal_vars] = target_encoder.fit_transform(df[nominal_vars], df['DefaultStatus'])

#Define the continuous variables
num_vars = ['Age', 'Number_of_Dependants', 'Product_Quantity', 'Total_Amount']

#Standard Scaler--Best for algorithms that assume normality eg linear models, SVM, Logistic Reg...it keeps outliers and centers data 
from sklearn.preprocessing import StandardScaler as stds
scaler = stds()
df[num_vars] = scaler.fit_transform(df[num_vars])

#MinMax Scaler--For algorithms that rely on distances or are sensitive to e scale eg KNN, Neural nets(nb use it when you dont have extreme outliers)
#from sklearn.preprocessing import MinMaxScaler as mms
#scaler = mms()
df[num_vars] = scaler.fit_transform(df[num_vars])

#Use Quantile transformer--robust for handling both scale and outliers
#from sklearn.preprocessing import QuantileTransformer as qt
#scaler = qt(n_quantiles=100)
#df[num_vars] = scaler.fit_transform(df[num_vars])


#Data Splitting
from sklearn.model_selection import train_test_split as tts
x = df.drop('DefaultStatus', axis = 1)
y = df['DefaultStatus']
x_train, x_test, y_train, y_test = tts(x, y, test_size = 0.3, random_state = 42)


In [ ]:
df.head

In [ ]:

###SUPERVISED MACHINE LEARNING METHODS

#Logistic Regression
from sklearn.linear_model import LogisticRegression as lg
model = lg(class_weight={0:1, 1:2}, max_iter=1000)
model.fit( x_train, y_train)


#Make predictions
predictions = model.predict(x_test)
from sklearn.metrics import accuracy_score as acs
score = acs(y_test, predictions)

#Odds Ratios
import numpy as np
odds_ratios = pd.Series(np.exp(model.coef_[0]),
index = x.columns)
print(odds_ratios.sort_values(ascending = False))

#Baseline log_odds
print('Intercept (log-odds):',
      model.intercept_[0])
print('Intercept (odds):',
      np.exp(model.intercept_[0]))


#Evaluation metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
predictions = model.predict(x_test)
probs = model.predict_proba(x_test)[:, 1]

#Classification report
print('\nClassification Report:')
print('Report:',classification_report(y_test, predictions))

#Confusion matrix
print('\nConfusion matrix:')
print('Confusion:', confusion_matrix(y_test, predictions))

#ROC Curve
fpr, tpr, _ = roc_curve(y_test, probs)
plt.plot(fpr, tpr,
         label = 'Logistic Regression')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curve')
plt.legend()
plt.grid()
plt.show()

print('Accuracy:', score)

In [ ]:
#DECISION TREE
from sklearn.tree import DecisionTreeClassifier as dt
model1 = dt(random_state = 42)
model1.fit(x_train, y_train)


#Evaluation metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score as acs
predictions = model1.predict(x_test)
probs = model1.predict_proba(x_test)[:, 1]

#Classification report
print('\nClassification Report:')
print('Report:',classification_report(y_test, predictions))

#Confusion matrix
print('\nConfusion matrix:')
print('Confusion:', confusion_matrix(y_test, predictions))

#ROC Curve
fpr, tpr, _ = roc_curve(y_test, probs)
plt.plot(fpr, tpr,
         label = 'Decision Tree')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curve')
plt.legend()
plt.grid()
plt.show()


score = acs(y_test, predictions)

print('Accuracy:', score)

In [ ]:
#K-NEAREST NEIGHBORS
from sklearn.neighbors import KNeighborsClassifier as kn
model2 = kn(n_neighbors = 5)
model2.fit(x_train, y_train)


#Evaluation metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
predictions = model2.predict(x_test)
probs = model2.predict_proba(x_test)[:, 1]

#Classification report
print('\nClassification Report:')
print('Report:',classification_report(y_test, predictions))

#Confusion matrix
print('\nConfusion matrix:')
print('Confusion:', confusion_matrix(y_test, predictions))

#ROC Curve
fpr, tpr, _ = roc_curve(y_test, probs)
plt.plot(fpr, tpr,
         label = 'KNN')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curve')
plt.legend()
plt.grid()
plt.show()

from sklearn.metrics import accuracy_score as acs
score = acs(y_test, predictions)
print('Accuracy:', score)

In [ ]:
#Linear Discriminant Analysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as ld
model3 = ld()
model3.fit(x_train, y_train)


#Evaluation metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
predictions = model3.predict(x_test)
probs = model3.predict_proba(x_test)[:, 1]

#Classification report
print('\nClassification Report:')
print('Report:',classification_report(y_test, predictions))

#Confusion matrix
print('\nConfusion matrix:')
print('Confusion:', confusion_matrix(y_test, predictions))

#ROC Curve
fpr, tpr, _ = roc_curve(y_test, probs)
plt.plot(fpr, tpr,
         label = 'LDA')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curve')
plt.legend()
plt.grid()
plt.show()

from sklearn.metrics import accuracy_score as acs
score = acs(y_test, predictions)
print('Accuracy:', score)

In [ ]:
#Quadratic Discriminant Analysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as qd
model4 = qd()
model4.fit(x_train, y_train)


#Evaluation metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
predictions = model4.predict(x_test)
probs = model4.predict_proba(x_test)[:, 1]

#Classification report
print('\nClassification Report:')
print('Report:',classification_report(y_test, predictions))

#Confusion matrix
print('\nConfusion matrix:')
print('Confusion:', confusion_matrix(y_test, predictions))

#ROC Curve
fpr, tpr, _ = roc_curve(y_test, probs)
plt.plot(fpr, tpr,
         label = 'QDA')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curve')
plt.legend()
plt.grid()
plt.show()

from sklearn.metrics import accuracy_score as acs
score = acs(y_test, predictions)
print('Accuracy:', score)

In [ ]:
#SVM
from sklearn.svm import SVC as svm
model5 = svm(probability = True)
model5.fit(x_train, y_train)


#Evaluation metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
predictions = model5.predict(x_test)
probs = model5.predict_proba(x_test)[:, 1]

#Classification report
print('\nClassification Report:')
print('Report:',classification_report(y_test, predictions))

#Confusion matrix
print('\nConfusion matrix:')
print('Confusion:', confusion_matrix(y_test, predictions))

#ROC Curve
fpr, tpr, _ = roc_curve(y_test, probs)
plt.plot(fpr, tpr,
         label = 'SVM')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curve')
plt.legend()
plt.grid()
plt.show()

from sklearn.metrics import accuracy_score as acs
score = acs(y_test, predictions)
print('Accuracy:', score)

In [ ]:
#MLP
from sklearn.neural_network import MLPClassifier as mlp
model6 = mlp(hidden_layer_sizes=(100, 50), activation='tanh', solver='adam', random_state=42, max_iter=300)
model6.fit(x_train, y_train)


#Evaluation metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
predictions = model6.predict(x_test)
probs = model6.predict_proba(x_test)[:, 1]

#Classification report
print('\nClassification Report:')
print('Report:',classification_report(y_test, predictions))

#Confusion matrix
print('\nConfusion matrix:')
print('Confusion:', confusion_matrix(y_test, predictions))

#ROC Curve
fpr, tpr, _ = roc_curve(y_test, probs)
plt.plot(fpr, tpr,
         label = 'MLP')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curve')
plt.legend()
plt.grid()
plt.show()

from sklearn.metrics import accuracy_score as acs
score = acs(y_test, predictions)
print('Accuracy:', score)

In [ ]:

#ESEMBLE LEARNING
#Bagging
#Random Forest

from sklearn.ensemble import RandomForestClassifier as rf
model10 = rf(n_estimators=100, random_state=42)
model10.fit(x_train, y_train)


#Evaluation metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
predictions = model10.predict(x_test)
probs = model10.predict_proba(x_test)[:, 1]

#Classification report
print('\nClassification Report:')
print('Report:',classification_report(y_test, predictions))

#Confusion matrix
print('\nConfusion matrix:')
print('Confusion:', confusion_matrix(y_test, predictions))

#ROC Curve
fpr, tpr, _ = roc_curve(y_test, probs)
plt.plot(fpr, tpr,
         label = 'rf')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curve')
plt.legend()
plt.grid()
plt.show()

from sklearn.metrics import accuracy_score as acs
score = acs(y_test, predictions)
print('Accuracy:', score)

In [ ]:
#ANOTHER METHOD OF BAGGING(THE MANUALY ONE)

from sklearn.ensemble import BaggingClassifier as bc
from sklearn.tree import DecisionTreeClassifier as dt

#Base estimator
base_dt = dt()

#Bagging Classifier
model11 = bc(estimator=base_dt, n_estimators=100, random_state=42)
model11.fit(x_train, y_train)


#Evaluation metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
predictions = model11.predict(x_test)
probs = model11.predict_proba(x_test)[:, 1]

#Classification report
print('\nClassification Report:')
print('Report:',classification_report(y_test, predictions))

#Confusion matrix
print('\nConfusion matrix:')
print('Confusion:', confusion_matrix(y_test, predictions))

#ROC Curve
fpr, tpr, _ = roc_curve(y_test, probs)
plt.plot(fpr, tpr,
         label = 'Bagging Classifier')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curve')
plt.legend()
plt.grid()
plt.show()

from sklearn.metrics import accuracy_score as acs
score = acs(y_test, predictions)
print('Accuracy:', score)

In [ ]:
#BAGGING(USING LOGISTIC)

from sklearn.ensemble import BaggingClassifier as bc
from sklearn.linear_model import LogisticRegression as lg

#Base estimator
base_dt = lg()

#Bagging Classifier
model12 = bc(estimator=base_dt, n_estimators=100, random_state=42)
model12.fit(x_train, y_train)


#Evaluation metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
predictions = model12.predict(x_test)
probs = model12.predict_proba(x_test)[:, 1]

#Classification report
print('\nClassification Report:')
print('Report:',classification_report(y_test, predictions))

#Confusion matrix
print('\nConfusion matrix:')
print('Confusion:', confusion_matrix(y_test, predictions))

#ROC Curve
fpr, tpr, _ = roc_curve(y_test, probs)
plt.plot(fpr, tpr,
         label = 'Bagging ClassifierLOG')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curve')
plt.legend()
plt.grid()
plt.show()

from sklearn.metrics import accuracy_score as acs
score = acs(y_test, predictions)
print('Accuracy:', score)

In [ ]:
#Boosting
#Gradient Boosting

#BAGGING(USING LOGISTIC)


from sklearn.ensemble import GradientBoostingClassifier as gb

model13 = gb(n_estimators=100, learning_rate=0.1, random_state=42)
model13.fit(x_train, y_train)


#Evaluation metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
predictions = model13.predict(x_test)
probs = model13.predict_proba(x_test)[:, 1]

#Classification report
print('\nClassification Report:')
print('Report:',classification_report(y_test, predictions))

#Confusion matrix
print('\nConfusion matrix:')
print('Confusion:', confusion_matrix(y_test, predictions))

#ROC Curve
fpr, tpr, _ = roc_curve(y_test, probs)
plt.plot(fpr, tpr,
         label = 'GB')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curve')
plt.legend()
plt.grid()
plt.show()

from sklearn.metrics import accuracy_score as acs
score = acs(y_test, predictions)
print('Accuracy:', score)

In [ ]:
#Boosting
#ADABOOST

from sklearn.ensemble import AdaBoostClassifier as ad
from sklearn.tree import DecisionTreeClassifier as dt

#Base weak learner
base_model = dt(max_depth = 1)


#AdaBoost Classifier
model14 = ad(estimator = base_model, n_estimators=50, learning_rate=1.0, random_state=42)
model14.fit(x_train, y_train)


#Evaluation metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
predictions = model14.predict(x_test)
probs = model14.predict_proba(x_test)[:, 1]

#Classification report
print('\nClassification Report:')
print('Report:',classification_report(y_test, predictions))

#Confusion matrix
print('\nConfusion matrix:')
print('Confusion:', confusion_matrix(y_test, predictions))

#ROC Curve
fpr, tpr, _ = roc_curve(y_test, probs)
plt.plot(fpr, tpr,
         label = 'ADA')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curve')
plt.legend()
plt.grid()
plt.show()

from sklearn.metrics import accuracy_score as acs
score = acs(y_test, predictions)
print('Accuracy:', score)

In [ ]:
#Meta Estimators

#Voting Classifier(trains many models and average(hardvoting)or use majority voting)
from sklearn.tree import DecisionTreeClassifier as dtc
from sklearn.linear_model import LogisticRegression as lg
from sklearn.neighbors import KNeighborsClassifier as knc
from sklearn.ensemble import VotingClassifier as vc
from sklearn.datasets import make_classification as mc
import matplotlib.pylab as plt

In [ ]:
clf1 = model4
clf2 = knc(n_neighbors = 5).fit(x_train, y_train)
clf3 = vc(estimators = [('clf1', clf1), ('clf2', clf2)],
          voting = 'soft',
          weights = (0.5, 0.5))

clf3.fit(x_train, y_train)


In [ ]:
#Which base model is better

from sklearn.metrics import accuracy_score as acs

predLR = model6.predict(x_test)
predKN = clf2.predict(x_test)
predVC = clf3.predict(x_test)

scoreLR = acs(y_test, predLR)
scoreKN = acs(y_test, predKN)
scoreVC = acs(y_test, predVC)
print('Accuracy of QD:', scoreLR)
print('Accuracy of KN:', scoreKN)
print('Accuracy of VC:', scoreVC)

In [ ]:
df.head

In [ ]:
#Saving the models so that the next time we use them, we wont train again, but we rather only load and use them
import joblib
joblib.dump(model, 'model.pkl')
joblib.dump(model1, 'model1.pkl')
joblib.dump(model2, 'model2.pkl')
joblib.dump(model3, 'model3.pkl')
joblib.dump(model4, 'model4.pkl')
joblib.dump(model5, 'model5.pkl')
joblib.dump(model6, 'model6.pkl')

joblib.dump(model10, 'model10.pkl')
joblib.dump(model11, 'model11.pkl')
joblib.dump(model12, 'model12.pkl')
joblib.dump(model13, 'model13.pkl')
joblib.dump(model14, 'model14.pkl')

In [ ]:
#Loading and using the trained mode...an example
import joblib
import pickle
import streamlit as st
LR = joblib.load('model.pkl')

def main():
    st.title('Credit Risk Prediction Model')

#Input Variables
#User inputs
age = st.number_input("Age", min_value=18, max_value=100, value=30)
marital_status = st.selectbox("Marital Status", ['Single', 'Married', 'Divorced'])  # example categories
employment_status = st.selectbox("Employment Status", ['Employed', 'Unemployed'])
level_of_education = st.selectbox("Level of Education", [0, 1, 2])  # ordinal values you used during encoding
residence_area = st.selectbox("Residence Area", [0, 1, 2])
home_ownership = st.selectbox("Home Ownership", ['Yes', 'No'])
dependents = st.number_input("Number of Dependants", min_value=0)
product_quantity = st.number_input("Product Quantity", min_value=0)
total_amount = st.number_input("Total Amount", min_value=0.0)

#Prepare input
input_df = pd.DataFrame({
    'Age': [age],
    'Marital_Status': [marital_status],
    'Employment_Status': [employment_status],
    'Level_of_Education': [level_of_education],
    'Residence_Area': [residence_area],
    'Home_Ownership': [home_ownership],
    'Number_of_Dependants': [dependents],
    'Product_Quantity': [product_quantity],
    'Total_Amount': [total_amount]
})

#Predict
if st.button("Predict"):
    prediction = model.predict(input_df)
    st.success(f"Prediction: {'Default' if prediction[0] == 1 else 'No Default'}")

In [ ]:
##UNSUPERVISED MACHINE LEARNING METHODS

# K-Means Clustering
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

#Now the response variable is not allowed so drop it
x = df.drop('DefaultStatus', axis = 1)

#With three clusters
kmeans = KMeans(n_clusters = 3, random_state = 42)
kmeans.fit(x)

labels = kmeans.labels_

#Add cluster labels to e DataFrame
x['Cluster'] = labels

#Visualize the clusters with PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
x_pca = pca.fit_transform(x.drop('Cluster', axis = 1))

plt.figure(figsize=(8, 5))
plt.scatter(x_pca[:, 0], x_pca[:, 1], c = x['Cluster'], cmap = 'viridis')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('K-Means Clustering')
plt.colorbar(label='Cluster')
plt.show